In [8]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import citypy module to determine city name from coordinates
from citipy import citipy
# import datetime
from datetime import datetime
import requests
from config import weather_api_key
import time
# Import linear regression dependency
from scipy.stats import linregress

In [9]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [10]:
# Create a list to hold city names
cities = []

# Identify the cities for each longitude and latitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If city is unique we will add it to the cities list
    if city not in cities:
        cities.append(city)
# Print city count to confirm
print(len(cities))


618


In [11]:
city_data = []
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging
print('Beginning data retrieval    ')
print('----------------------------')

# Create counters
record_count = 1
set_count = 1
counter = 0
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
     # Run an API request for each of the cities.
    
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        city_precip = city_weather['weather'][0]['main']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        try:
            rain_3h = city_weather['rain']['3h']
            rain_3h_inches = rain_3h / 25.4
        except:
            rain_3h_inches = 0
            pass
        try:
            snow_3h = city_weather['snow']['3h']
            snow_3h_inches = snow_3h / 25.4
        except:
            snow_3h_inches = 0
            pass
        
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather Description": city_weather_description,
                          "City Precip": city_precip,
                          "Rain in Inches": rain_3h_inches,
                          "Snow in Inches": snow_3h_inches})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
    
    #city_data[counter].update({"Rain in Inches":rain_3h_inches})
    #counter +=1          
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

Beginning data retrieval    
----------------------------
Processing Record 1 of Set 1 | albany
Processing Record 2 of Set 1 | sola
Processing Record 3 of Set 1 | lagoa
Processing Record 4 of Set 1 | east london
Processing Record 5 of Set 1 | chake chake
Processing Record 6 of Set 1 | arraial do cabo
Processing Record 7 of Set 1 | sept-iles
Processing Record 8 of Set 1 | mergui
City not found. Skipping...
Processing Record 9 of Set 1 | hilo
Processing Record 10 of Set 1 | georgetown
Processing Record 11 of Set 1 | nikolskoye
Processing Record 12 of Set 1 | formoso do araguaia
City not found. Skipping...
Processing Record 13 of Set 1 | khuzdar
Processing Record 14 of Set 1 | atuona
Processing Record 15 of Set 1 | camacha
Processing Record 16 of Set 1 | kaitangata
Processing Record 17 of Set 1 | faanui
Processing Record 18 of Set 1 | punta arenas
Processing Record 19 of Set 1 | upernavik
Processing Record 20 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 21 of Set 1 |

In [12]:
city_data

[{'City': 'Albany',
  'Lat': 42.6,
  'Lng': -73.97,
  'Max Temp': 53.01,
  'Humidity': 45,
  'Cloudiness': 90,
  'Wind Speed': 6.93,
  'Country': 'US',
  'Date': '2020-04-01 20:29:53',
  'Weather Description': 'overcast clouds',
  'City Precip': 'Clouds',
  'Rain in Inches': 0,
  'Snow in Inches': 0},
 {'City': 'Sola',
  'Lat': -13.88,
  'Lng': 167.55,
  'Max Temp': 82.94,
  'Humidity': 77,
  'Cloudiness': 97,
  'Wind Speed': 12.33,
  'Country': 'VU',
  'Date': '2020-04-01 20:33:17',
  'Weather Description': 'overcast clouds',
  'City Precip': 'Clouds',
  'Rain in Inches': 0,
  'Snow in Inches': 0},
 {'City': 'Lagoa',
  'Lat': 39.05,
  'Lng': -27.98,
  'Max Temp': 57.27,
  'Humidity': 80,
  'Cloudiness': 100,
  'Wind Speed': 30.38,
  'Country': 'PT',
  'Date': '2020-04-01 20:33:17',
  'Weather Description': 'light rain',
  'City Precip': 'Rain',
  'Rain in Inches': 0.08346456692913387,
  'Snow in Inches': 0},
 {'City': 'East London',
  'Lat': -33.02,
  'Lng': 27.91,
  'Max Temp': 66.06